In [ ]:
from google.colab import drive #to connect google drive to the code
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

# 데이터 전처리
- 양품 : 75 * 4 = 300
- 불량품 : 300 * 4 = 1200 랜덤 샘플링

In [ ]:
import pandas as pd
import ast

# CSV 파일 경로
input_csv = "./bumper_data.csv"
output_csv = "./bumper_data.csv"

# CSV 파일 읽기
df = pd.read_csv(input_csv)

# bboxes 열의 문자열을 리스트로 변환 (필요 시)
def parse_bboxes(bboxes_str):
    try:
        return ast.literal_eval(bboxes_str)
    except (ValueError, SyntaxError):
        return []

df["bboxes"] = df["bboxes"].apply(parse_bboxes)

# 중복 제거 (path 열 기준)
df_unique = df.drop_duplicates(subset="path")

# 정렬 (quality 열 기준, 내림차순)
df_sorted = df_unique.sort_values(by="quality", ascending=False)

# bboxes를 다시 문자열로 변환하여 저장
df_sorted["bboxes"] = df_sorted["bboxes"].apply(str)

# 정리된 데이터 저장
df_sorted.to_csv(output_csv, index=False)

print(f"중복 제거 및 정렬된 CSV 파일이 저장되었습니다: {output_csv}")


중복 제거 및 정렬된 CSV 파일이 저장되었습니다: ./bumper_data.csv


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./bumper_data.csv')



# 0이 처음 출력되는 index 값 추출
first_zero_bumper = data_bumper[data_bumper['quality'] == 0].index[0]




In [ ]:
data_bumper.head()

,path,quality,part,bboxes
0,/content/drive/MyDrive/bumper/images/205_101_1...,1,범퍼,[]
1,/content/drive/MyDrive/bumper/images/205_101_1...,1,범퍼,[]
2,/content/drive/MyDrive/bumper/images/205_101_1...,1,범퍼,[]
3,/content/drive/MyDrive/bumper/images/205_101_1...,1,범퍼,[]
4,/content/drive/MyDrive/bumper/images/205_101_1...,1,범퍼,[]


# 일부 에러 데이터 정규화

In [ ]:
# import pandas as pd

# def modify_csv(input_file, output_file, column_name, new_value, first_zero):
#     # CSV 파일 읽기
#     df = pd.read_csv(input_file)

#     # 조건 설정: 첫 번째 0이 나오기 전까지만
#     row_condition = df.index < first_zero_frame_hd

#     # 조건에 해당하는 값 수정
#     df.loc[row_condition, column_name] = new_value

#     # 수정된 데이터 저장
#     df.to_csv(output_file, index=False)

# # 함수 사용 예시
# # modify_csv(
# #     "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\data\\data\\output\\frame\\frame_hd_data.csv",
# #     "./data/data/output/frame/frame_hd_data_modify.csv",
# #     "bboxes",
# #     "[]",
# #     first_zero_frame_hd
# # )

# # modify_csv(
# #     "./data/data/output/frame/frame_sealf_data.csv",
# #     "./data/data/output/frame/frame_sealf_data_modify.csv",
# #     "bboxes",
# #     "[]",
# #     first_zero_frame_sealf
# # )

# # modify_csv(
# #     "./data/data/output/frame/frame_seamf_data.csv",
# #     "./data/data/output/frame/frame_seamf_data_modify.csv",
# #     "bboxes",
# #     "[]",
# #     first_zero_frame_seamf
# # )


### index을 통해서 양품과 불량품의 위치를 구분

In [ ]:
# print(f"bumper 양품 : {len(data_bumper.iloc[:first_zero_bumper])}")
# print(f"bumper 불량품 : {len(data_bumper.iloc[first_zero_bumper:])}")

# print(f"door_ed 양품 : {len(data_door_ed.iloc[:first_zero_door_ed])}")
# print(f"door_ed 불양품 : {len(data_door_ed.iloc[first_zero_door_ed:])}")
# print(f"door_scratch 양품 : {len(data_door_scratch.iloc[:first_zero_door_scratch])}")
# print(f"door_scratch 불양품 : {len(data_door_scratch.iloc[first_zero_door_scratch:])}")

# print(f"data_fender 양품 : {len(data_fender.iloc[:first_zero_fender])}")
# print(f"data_fender 불량품 : {len(data_fender.iloc[first_zero_fender:])}")

# print(f"data_frame_ed 양품 : {len(data_frame_ed.iloc[:first_zero_frame_ed])}")
# print(f"data_frame_ed 불양품 : {len(data_frame_ed.iloc[first_zero_frame_ed:])}")
# print(f"data_frame_hd 양품 : {len(data_frame_hd.iloc[:first_zero_frame_hd])}")
# print(f"data_frame_hd 불양품 : {len(data_frame_hd.iloc[first_zero_frame_hd:])}")
# print(f"data_frame_sealf 양품 : {len(data_frame_sealf.iloc[:first_zero_frame_sealf])}")
# print(f"data_frame_sealf 불양품 : {len(data_frame_sealf.iloc[first_zero_frame_sealf:])}")
# print(f"data_frame_seamf 양품 : {len(data_frame_seamf.iloc[:first_zero_frame_seamf])}")
# print(f"data_frame_seamf 불양품 : {len(data_frame_seamf.iloc[first_zero_frame_seamf:])}")



### 양품 데이터 추출

In [ ]:
import pandas as pd

bumper_before_zero = data_bumper.iloc[:first_zero_bumper]



# 75개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_ok_random_sample = bumper_before_zero.sample(
    n=min(1400, len(bumper_before_zero)), replace=False, random_state=55)

# 결과 출력
# print(bumper_ok_random_sample)
# print(len(bumper_ok_random_sample))

# print(fender_ok_random_sample)
# print(len(fender_ok_random_sample))

# print(door_ok_random_sample)
# print(len(door_ok_random_sample))

# print(frame_ok_random_sample)
# print(len(frame_ok_random_sample))

ok_data = pd.concat([bumper_ok_random_sample], ignore_index=True, axis=0)

# ok_data

### 불량품 데이터 추출

In [ ]:
import pandas as pd
data_fault_bumper = data_bumper.iloc[first_zero_bumper:]



# 300개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_fault_random_sample = data_fault_bumper.sample(n=1200, replace=False)

# 결과 출력
# print(bumper_fault_random_sample)
# print(len(bumper_fault_random_sample))

# print(fender_fault_random_sample)
# print(len(fender_fault_random_sample))

# print(door_fault_random_sample)
# print(len(door_fault_random_sample))

# print(frame_fault_random_sample)
# print(len(frame_fault_random_sample))

fault_data = pd.concat([bumper_fault_random_sample], ignore_index=True, axis=0)

# fault_data

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# y에는 라벨, x에는 나머지 특성 정보
y_ok = ok_data['quality']
y_fault = fault_data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data_ok, test_data_ok = train_test_split(
    ok_data, test_size=0.2, stratify=y_ok, random_state=60
)
train_data_fault, test_data_fault = train_test_split(
    fault_data, test_size=0.2, stratify=y_fault, random_state=60
)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80_ok, valid_data_ok = train_test_split(
    train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
train_data_80_fault, valid_data_fault = train_test_split(
    train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# CSV로 저장
train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)

# 결과 출력
print(f"y_ok 총 데이터 개수: {len(y_ok)}")
print(f"y_fault 총 데이터 개수: {len(y_fault)}")
print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
print(f"train_data_80_fault 학습 데이터 개수: {len(train_data_80_fault)}")
print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


y_ok 총 데이터 개수: 1400
y_fault 총 데이터 개수: 1500
train_data_80_ok 학습 데이터 개수: 896
train_data_80_fault 학습 데이터 개수: 960
test_data_ok 테스트 데이터 개수: 280
test_data_fault 테스트 데이터 개수: 300
valid_data_ok 검증 데이터 개수: 224
valid_data_fault 검증 데이터 개수: 240


# 테스트 데이터

### bboxes 데이터 추출

In [ ]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_ok_bboxes = parse_bboxes(train_data_80_ok)
valid_ok_bboxes = parse_bboxes(valid_data_ok)
test_ok_bboxes = parse_bboxes(test_data_ok)
train_fault_bboxes = parse_bboxes(train_data_80_fault)
valid_fault_bboxes = parse_bboxes(valid_data_fault)
test_fault_bboxes = parse_bboxes(test_data_fault)

# 결과 출력 (예시)
# print(train_ok_bboxes[:5])
# print(valid_ok_bboxes[:5])
# print(test_ok_bboxes[:5])
# print(train_fault_bboxes[:5])
# print(valid_fault_bboxes[:5])
# print(test_fault_bboxes[:5])

# .yaml label 데이터 생성

### 1. 디렉토리 경로 전처리

In [ ]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_ok_image_paths = train_data_80_ok.iloc[:, 0].tolist()
valid_ok_image_paths = valid_data_ok.iloc[:, 0].tolist()
test_ok_image_paths = test_data_ok.iloc[:, 0].tolist()
train_fault_image_paths = train_data_80_fault.iloc[:, 0].tolist()
valid_fault_image_paths = valid_data_fault.iloc[:, 0].tolist()
test_fault_image_paths = test_data_fault.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes_ok, train_dirs_ok, train_names_ok = extract_image_info(train_ok_image_paths)
valid_sizes_ok, valid_dirs_ok, valid_names_ok = extract_image_info(valid_ok_image_paths)
test_sizes_ok, test_dirs_ok, test_names_ok = extract_image_info(test_ok_image_paths)
train_sizes_fault, train_dirs_fault, train_names_fault = extract_image_info(train_fault_image_paths)
valid_sizes_fault, valid_dirs_fault, valid_names_fault = extract_image_info(valid_fault_image_paths)
test_sizes_fault, test_dirs_fault, test_names_fault = extract_image_info(test_fault_image_paths)

# 결과 출력 ok
# print("train_sizes_ok 이미지 크기:", train_sizes_ok[:5])
# print("train_dirs_ok 이미지 디렉토리:", train_dirs_ok[:5])
# print("train_names_ok 파일 이름:", train_names_ok[:5])
# print()
# print("valid_sizes_ok 이미지 크기:", valid_sizes_ok[:5])
# print("valid_dirs_ok 이미지 디렉토리:", valid_dirs_ok[:5])
# print("valid_names_ok 파일 이름:", valid_names_ok[:5])
# print()
# print("test_sizes_ok 이미지 크기:", test_sizes_ok[:5])
# print("test_dirs_ok 이미지 디렉토리:", test_dirs_ok[:5])
# print("test_names_ok 파일 이름:", test_names_ok[:5])
# 결과 출력 fault
# print("train_sizes_fault 이미지 크기:", train_sizes_fault[:5])
# print("train_dirs_fault 이미지 디렉토리:", train_dirs_fault[:5])
# print("train_names_fault 파일 이름:", train_names_fault[:5])
# print()
# print("valid_sizes_fault 이미지 크기:", valid_sizes_fault[:5])
# print("valid_dirs_fault 이미지 디렉토리:", valid_dirs_fault[:5])
# print("valid_names_fault 파일 이름:", valid_names_fault[:5])
# print()
# print("test_sizes_fault 이미지 크기:", test_sizes_fault[:5])
# print("test_dirs_fault 이미지 디렉토리:", test_dirs_fault[:5])
# print("test_names_fault 파일 이름:", test_names_fault[:5])

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


### 3. 부품 상태 전처리

In [ ]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    return None  # 기본적으로 반환할 값이 없으면 None

# 예시 부품과 상태
parts_and_status_train_ok = []
parts_and_status_valid_ok = []
parts_and_status_test_ok = []
for quality, part in zip(train_data_80_ok.iloc[:, 1], train_data_80_ok.iloc[:, 2]):
    parts_and_status_train_ok.append({"part": part, "status": quality})

for quality, part in zip(test_data_ok.iloc[:, 1], test_data_ok.iloc[:, 2]):
    parts_and_status_test_ok.append({"part": part, "status": quality})

for quality, part in zip(valid_data_ok.iloc[:, 1], valid_data_ok.iloc[:, 2]):
    parts_and_status_valid_ok.append({"part": part, "status": quality})

parts_and_status_train_fault = []
parts_and_status_valid_fault = []
parts_and_status_test_fault = []
for quality, part in zip(train_data_80_fault.iloc[:, 1], train_data_80_fault.iloc[:, 2]):
    parts_and_status_train_fault.append({"part": part, "status": quality})

for quality, part in zip(test_data_fault.iloc[:, 1], test_data_fault.iloc[:, 2]):
    parts_and_status_test_fault.append({"part": part, "status": quality})

for quality, part in zip(valid_data_fault.iloc[:, 1], valid_data_fault.iloc[:, 2]):
    parts_and_status_valid_fault.append({"part": part, "status": quality})

# print(f"parts_and_status_train : {parts_and_status_train_ok}")
# print(f"parts_and_status_test : {parts_and_status_test_ok}")
# print(f"parts_and_status_valid : {parts_and_status_valid_ok}")

# print(f"parts_and_status_train : {parts_and_status_train_fault}")
# print(f"parts_and_status_test : {parts_and_status_test_fault}")
# print(f"parts_and_status_valid : {parts_and_status_valid_fault}")

# 라벨링 결과를 저장할 리스트
labels_train_ok = []
labels_valid_ok = []
label_test_ok = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_ok:
    label = get_label(item["part"], item["status"])
    labels_train_ok.append(label)
for item in parts_and_status_test_ok:
    label = get_label(item["part"], item["status"])
    label_test_ok.append(label)
for item in parts_and_status_valid_ok:
    label = get_label(item["part"], item["status"])
    labels_valid_ok.append(label)

# # 결과 출력
# print("labels_train_ok 라벨링 결과 :", labels_train_ok)
# print("labels_valid_ok 라벨링 결과 :", labels_valid_ok)
# print("label_test_ok 라벨링 결과 :", label_test_ok)

# 라벨링 결과를 저장할 리스트
labels_train_fault = []
labels_valid_fault = []
label_test_fault = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_fault:
    label = get_label(item["part"], item["status"])
    labels_train_fault.append(label)
for item in parts_and_status_test_fault:
    label = get_label(item["part"], item["status"])
    label_test_fault.append(label)
for item in parts_and_status_valid_fault:
    label = get_label(item["part"], item["status"])
    labels_valid_fault.append(label)

# # 결과 출력
# print("labels_train_fault 라벨링 결과 :", labels_train_fault)
# print("labels_valid_fault 라벨링 결과 :", labels_valid_fault)
# print("label_test_fualt 라벨링 결과 :", label_test_fault)

In [ ]:
import os
import shutil

# 폴더 초기화 함수
def prepare_output_dir(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # 기존 폴더 삭제
    os.makedirs(output_dir, exist_ok=True)  # 새 폴더 생성

# 이미지 복사 함수
def copy_images(output_dir, data):
    image_dir = os.path.join(output_dir, 'images')

    # 이미지 폴더가 존재하는지 확인하고, 없으면 새로 생성
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    for index, row in data.iterrows():
        image_path = row['path']
        if os.path.exists(image_path):  # 이미지 경로 확인
            shutil.copy(image_path, image_dir)
        else:
            print(f"[WARNING] Image not found: {image_path}")

# YOLO 형식 변환 함수
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []

    if not bbox or len(bbox) == 0:  # 바운딩 박스가 없을 경우
        yolo_labels.append(f"{label} 0.0 0.0 0.0 0.0")
    else:
        for x_min, y_min, width, height in bbox:
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

    return yolo_labels

# YOLO 라벨 저장 함수
def save_yolo_labels(output_dir, bboxes, labels, size, names):
    label_dir = os.path.join(output_dir, 'labels')

    # 라벨 폴더가 존재하는지 확인하고, 없으면 새로 생성
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        txt_file_path = os.path.join(label_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")

        # 디버깅 로그 추가
        print(f"[INFO] Saved YOLO label: {txt_file_path} (bbox count: {len(bbox)})")

# 이미지 복사 및 라벨 저장 작업을 한 번만 처리하도록 수정
def process_data(output_dir, data, bboxes, labels, sizes, names):

    copy_images(output_dir, data)  # 이미지 복사
    save_yolo_labels(output_dir, bboxes, labels, sizes, names)  # YOLO 라벨 저장

# 디렉토리 설정
train_output_dir = '/content/drive/MyDrive/train'
valid_output_dir = '/content/drive/MyDrive/valid'
test_output_dir = '/content/drive/MyDrive/test'
prepare_output_dir(train_output_dir)
prepare_output_dir(valid_output_dir)
prepare_output_dir(test_output_dir)

# 이미지 및 라벨 처리
process_data(train_output_dir, train_data_80_ok, train_ok_bboxes, labels_train_ok, train_sizes_ok, train_names_ok)
process_data(test_output_dir, test_data_ok, test_ok_bboxes, label_test_ok, test_sizes_ok, test_names_ok)
process_data(valid_output_dir, valid_data_ok, valid_ok_bboxes, labels_valid_ok, valid_sizes_ok, valid_names_ok)

process_data(train_output_dir, train_data_80_fault, train_fault_bboxes, labels_train_fault, train_sizes_fault, train_names_fault)
process_data(test_output_dir, test_data_fault, test_fault_bboxes, label_test_fault, test_sizes_fault, test_names_fault)
process_data(valid_output_dir, valid_data_fault, valid_fault_bboxes, labels_valid_fault, valid_sizes_fault, valid_names_fault)


[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_7293281f-e0cc-4897-a9c8-3ea6b2b00844.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_0e64f458-067a-465c-a640-0f9cb9ac2151.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_49963a85-ffb2-49b4-b815-f03f553a962c.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_4a2f24cc-3b70-43c1-8c75-55025cf71216.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_4ca2e2b0-f655-40f5-b6e2-54f19776bac5.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_11e4a7b4-01cc-45e4-a8d0-d8fa644f1de4.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_41fee01f-9d47-45f2-bad0-85ff11ecffef.txt (bbox count: 0)
[INFO] Saved YOLO label: /content/drive/MyDrive/train/labels/205_101_10_227a2664-ca8b-411c-a60c-a

## 모델 코드 작성

In [ ]:
import yaml

# YAML 파일에 저장할 데이터
yaml_data = {
    "train": "/content/drive/MyDrive/train/images",
    "test": "/content/drive/MyDrive/test/images",
    "val": "/content/drive/MyDrive/valid/images",
    "nc": 2,
    "names": ["bumper_ok", "bumper_faulty",],
    "augment": {
        "flipud": 0.3,
        "fliplr": 0.3,
        "rotate": 5,  # 회전 범위 줄이기
        "mosaic": 0.5,  # 모자이크 비율 줄이기
        "translate": 0.2,
        "scale": 0.7,
        "random_shadow": 0.4,  # 그림자 효과를 증가시켜 밝기 변화에 강하게
        "random_fog": 0.3,  # 안개 효과를 늘려서 다양한 시나리오 학습
        "crop": 0.3,  # 잘라내기 영역을 증가시켜 모델의 일반화 성능 향상
        "brightness": 0.4,  # 밝기 변화 범위 증가
        "contrast": 0.4,  # 대비 범위 증가
        "sharpen": 0.3,  # 이미지 샤프닝 강도를 높임
        "mixup": 0.2,  # mixup 비율 줄이기
        "cutmix": 0.1,  # cutmix 비율 줄이기
        "motion_blur": 0.2,  # motion blur 적용 강도 줄이기
        "gauss_noise": 0.2,  # 노이즈 강도 줄이기
        "hsv_h": 0.015,  # 색조 변화 증가
        "hsv_s": 0.7,  # 채도 변화 증가
        "hsv_v": 0.4,  # 명도 변화 증가
        "shear": 20,  # 전단 변환 강도 증가
        "perspective": 0.2,  # 투시 변환 추가
    },
}

# YAML 파일 생성
with open("data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

print("YAML 파일이 성공적으로 생성되었습니다.")

YAML 파일이 성공적으로 생성되었습니다.


In [ ]:

pip install ultralytics

In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=200,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=12,  # 배치 크기 늘리기
    imgsz=640,  # 입력 이미지 크기 512로 조정 (성능 및 메모리 효율 고려)
    workers=8,  # 데이터 로드 병렬 처리 워커 수
    project="/content/drive/MyDrive",
    name="model_8_s_200_12_640",
    lr0=0.0001,  # 초기 학습률
    lrf=0.1,     # 학습률 범위
    conf=0.3,    # Confidence threshold
    augment=True,  # 데이터 증강 활성화
    optimizer="Adam",  # AdamW 옵티마이저 사용
    cos_lr=True,  # 코사인 학습률 스케줄링
    momentum=0.6,  # 모멘텀
    cache=True,  # 데이터 로딩 속도 개선
)

# 학습 완료 후 모델 저장
model.save('model_8_s_200_12_640.pt')


Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=200, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive, name=model_8_s_200_12_6403, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.3, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/drive/MyDrive/train/labels.cache... 1855 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1856/1856 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08bfceb6-adaf-4a7a-b0f4-713127fdff31.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08c5a755-1036-4f06-b89d-a7b0daf7d057.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_093135c1-e887-4ce2-93a7-d3cd71316024.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_0957d6af-5a46-43cf-9d96-00c19be856bc.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09ab3429-efae-478d-81f7-12afe41dc528.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09c5578f-1578-4e77-b12b-598c56173ac7.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09f0ec44-ae5b-412f-9bc8-8e8075224e0c.jpg: corrupt JPEG

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.2GB RAM): 100%|██████████| 1855/1855 [00:38<00:00, 48.73it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    158/200      2.98G     0.3424      0.258     0.8344         15        640: 100%|██████████| 155/155 [00:30<00:00,  5.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:02<00:00,  8.84it/s]


                   all        464        658      0.308      0.281      0.296       0.19
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 58, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

158 epochs completed in 1.489 hours.
Optimizer stripped from /content/drive/MyDrive/model_8_s_200_12_6403/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/model_8_s_200_12_6403/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/model_8_s_200_12_6403/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.16it/s]


                   all        464        658      0.302      0.288      0.295      0.195
             bumper_ok        224        224          0          0          0          0
         bumper_faulty        240        434      0.603      0.575       0.59      0.389
Speed: 0.1ms preprocess, 6.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/model_8_s_200_12_6403


In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=100,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=12,  # 배치 크기 늘리기
    imgsz=640,  # 입력 이미지 크기 512로 조정 (성능 및 메모리 효율 고려)
    workers=8,  # 데이터 로드 병렬 처리 워커 수
    project="/content/drive/MyDrive",
    name="model_8_s_300_12_640",
    lr0=0.0001,  # 초기 학습률
    lrf=0.1,     # 학습률 범위
    conf=0.2,    # Confidence threshold
    augment=True,  # 데이터 증강 활성화
    optimizer="Adam",  # AdamW 옵티마이저 사용
    cos_lr=True,  # 코사인 학습률 스케줄링
    momentum=0.6,  # 모멘텀
    cache=True,  # 데이터 로딩 속도 개선
)

# 학습 완료 후 모델 저장
model.save('model_8_s_100_12_640_0.2.pt')


In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=300,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=12,  # 배치 크기 늘리기
    imgsz=640,  # 입력 이미지 크기 512로 조정 (성능 및 메모리 효율 고려)
    workers=8,  # 데이터 로드 병렬 처리 워커 수
    project="/content/drive/MyDrive",
    name="model_8_s_300_12_640",
    lr0=0.0001,  # 초기 학습률
    lrf=0.1,     # 학습률 범위
    conf=0.25,    # Confidence threshold
    augment=True,  # 데이터 증강 활성화
    optimizer="Adam",  # AdamW 옵티마이저 사용
    cos_lr=True,  # 코사인 학습률 스케줄링
    momentum=0.6,  # 모멘텀
    cache=True,  # 데이터 로딩 속도 개선
)

# 학습 완료 후 모델 저장
model.save('model_8_s_200_12_640.pt')


Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=300, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive, name=model_8_s_300_12_640, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.25, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/drive/MyDrive/train/labels.cache... 1855 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1856/1856 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08bfceb6-adaf-4a7a-b0f4-713127fdff31.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08c5a755-1036-4f06-b89d-a7b0daf7d057.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_093135c1-e887-4ce2-93a7-d3cd71316024.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_0957d6af-5a46-43cf-9d96-00c19be856bc.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09ab3429-efae-478d-81f7-12afe41dc528.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09c5578f-1578-4e77-b12b-598c56173ac7.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09f0ec44-ae5b-412f-9bc8-8e8075224e0c.jpg: corrupt JPEG

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.2GB RAM): 100%|██████████| 1855/1855 [00:37<00:00, 49.49it/s]

...

    195/300      2.99G     0.3395     0.2554     0.8316         26        640: 100%|██████████| 155/155 [00:30<00:00,  5.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:02<00:00,  8.96it/s]

                   all        464        658      0.313      0.291      0.296      0.186
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 95, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



195 epochs completed in 1.841 hours.
Optimizer stripped from /content/drive/MyDrive/model_8_s_300_12_640/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/model_8_s_300_12_640/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/model_8_s_300_12_640/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.10it/s]


                   all        464        658      0.293      0.296      0.295      0.192
             bumper_ok        224        224          0          0          0          0
         bumper_faulty        240        434      0.586      0.592      0.589      0.385
Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/model_8_s_300_12_640


In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=300,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=12,  # 배치 크기 늘리기
    imgsz=640,  # 입력 이미지 크기 512로 조정 (성능 및 메모리 효율 고려)
    workers=8,  # 데이터 로드 병렬 처리 워커 수
    project="/content/drive/MyDrive",
    name="model_8_s_300_12_640",
    lr0=0.0001,  # 초기 학습률
    lrf=0.1,     # 학습률 범위
    conf=0.4,    # Confidence threshold
    augment=True,  # 데이터 증강 활성화
    optimizer="Adam",  # AdamW 옵티마이저 사용
    cos_lr=True,  # 코사인 학습률 스케줄링
    momentum=0.6,  # 모멘텀
    cache=True,  # 데이터 로딩 속도 개선
)

# 학습 완료 후 모델 저장
model.save('model_8_s_300_12_640.pt')


Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=300, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive, name=model_8_s_300_12_6402, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.4, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/drive/MyDrive/train/labels.cache... 1855 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1856/1856 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08bfceb6-adaf-4a7a-b0f4-713127fdff31.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08c5a755-1036-4f06-b89d-a7b0daf7d057.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_093135c1-e887-4ce2-93a7-d3cd71316024.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_0957d6af-5a46-43cf-9d96-00c19be856bc.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09ab3429-efae-478d-81f7-12afe41dc528.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09c5578f-1578-4e77-b12b-598c56173ac7.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09f0ec44-ae5b-412f-9bc8-8e8075224e0c.jpg: corrupt JPEG

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.2GB RAM): 100%|██████████| 1855/1855 [00:37<00:00, 49.86it/s]

...

    234/300      2.99G     0.3053     0.2297     0.8207         20        640: 100%|██████████| 155/155 [00:30<00:00,  5.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:02<00:00,  8.55it/s]

                   all        464        658      0.337      0.262      0.286      0.186
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 134, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



234 epochs completed in 2.207 hours.
Optimizer stripped from /content/drive/MyDrive/model_8_s_300_12_6402/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/model_8_s_300_12_6402/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/model_8_s_300_12_6402/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.19it/s]


                   all        464        658      0.309      0.287      0.296      0.197
             bumper_ok        224        224          0          0          0          0
         bumper_faulty        240        434      0.619      0.574      0.593      0.393
Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/model_8_s_300_12_6402


In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=300,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=12,  # 배치 크기 늘리기
    imgsz=960,  # 입력 이미지 크기 512로 조정 (성능 및 메모리 효율 고려)
    workers=8,  # 데이터 로드 병렬 처리 워커 수
    project="/content/drive/MyDrive",
    name="model_8_s_300_12_640",
    lr0=0.0001,  # 초기 학습률
    lrf=0.1,     # 학습률 범위
    conf=0.3,    # Confidence threshold
    augment=True,  # 데이터 증강 활성화
    optimizer="Adam",  # AdamW 옵티마이저 사용
    cos_lr=True,  # 코사인 학습률 스케줄링
    momentum=0.6,  # 모멘텀
    cache=True,  # 데이터 로딩 속도 개선
)

# 학습 완료 후 모델 저장
model.save('model_8_s_300_12_960.pt')


Ultralytics 8.3.66 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=300, time=None, patience=100, batch=12, imgsz=960, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive, name=model_8_s_300_12_6403, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.3, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/drive/MyDrive/train/labels.cache... 1855 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1856/1856 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08bfceb6-adaf-4a7a-b0f4-713127fdff31.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_08c5a755-1036-4f06-b89d-a7b0daf7d057.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_093135c1-e887-4ce2-93a7-d3cd71316024.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_0957d6af-5a46-43cf-9d96-00c19be856bc.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09ab3429-efae-478d-81f7-12afe41dc528.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09c5578f-1578-4e77-b12b-598c56173ac7.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/train/images/205_101_10_09f0ec44-ae5b-412f-9bc8-8e8075224e0c.jpg: corrupt JPEG

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.7GB RAM): 100%|██████████| 1855/1855 [00:38<00:00, 47.65it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     85/300      6.54G     0.4077     0.3068     0.8674         31        960: 100%|██████████| 155/155 [01:05<00:00,  2.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.02it/s]

                   all        464        658      0.311      0.278      0.287      0.181



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     86/300      6.53G     0.4158     0.3112     0.8684         45        960:  59%|█████▉    | 92/155 [00:39<00:26,  2.34it/s]

In [ ]:
from ultralytics import YOLO
# 모델 로드 및 추론
model = YOLO("C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_11_s_200_12_576.pt")
results = model.predict(source="C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\test\\images", save=True, imgsz=640)


image 1/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_0107f873-8e8a-43d6-94c5-703a77b87e08.jpg: 320x640 (no detections), 150.3ms
image 2/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_01ba8ed7-8669-4af9-ad2f-7c73a124e769.jpg: 480x640 (no detections), 96.1ms
image 3/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_0501f4ab-216c-4b39-bb2e-fd17f52b8143.jpg: 384x640 (no detections), 126.3ms
image 4/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_053a2d06-aec9-4456-b888-2787e38b3715.jpg: 288x640 (no detections), 84.2ms
image 5/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_084265be-6211-4393-acb7-e45b195d96d4.jpg: 320x640 (no detections), 34.5ms
image 6/180 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\205_101_10_0a4032d1-888a-4857-99af-601146fdd784.jpg: 480x640 (no detections), 52.2ms
image 7/180 C:\Users\dawoo\desktop\sf7\codi

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = []
test_test_20 = pd.concat([test_data_ok, test_data_fault], ignore_index=True)

for result in results:
    if len(result.boxes) > 0:  # Detection이 있을 경우
        # 각 box에서 confidence가 가장 높은 클래스 선택
        pred.append(int(result.boxes[0].cls))
    else:
        pred.append(0)  # No Detection은 '0'으로 처리


# y_test 생성
y_test = []

for i in range(len(test_test_20)):
    if test_test_20.iloc[i, 1] == 1:  # 열 1에 접근
        y_test.append(0)
    elif test_test_20.iloc[i, 2] == '범퍼':  # 열 2에 접근
        y_test.append(1)
    elif test_test_20.iloc[i, 2] == '도어':
        y_test.append(2)
    elif test_test_20.iloc[i, 2] == '휀더':
        y_test.append(3)
    elif test_test_20.iloc[i, 2] == '프레임':
        y_test.append(4)


# data evaluation


# 위의 ROC 커브 코드 재사용 가능

print(y_test)
print(pred)

# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat, sep='\n')


# 정확도
accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('정확도:', accuracy)

# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
print('재현율: ', recall)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 학습된 모델 경로
model = YOLO("model_11_s_200_12_576.pt")  # 예: 'model_8_m_100_8_640.pt'

# 이미지 파일 경로
image_path = "C:\\Users\\dawoo\\Desktop\\범퍼1.jpg"

# 이미지를 불러와서 예측
result = model(image_path)

# 예측 결과 출력
result[0].show()  # 첫 번째 예측 결과의 바운딩 박스를 이미지로 출력

# 예측된 객체들의 정보 출력
boxes = result[0].boxes.xywh  # 바운딩 박스 정보 (x_center, y_center, width, height)
print(f"boxes : {boxes}")  # 예측된 바운딩 박스 정보 출력

# 만약 이미지 저장을 원한다면:
result[0].save("C:\\Users\\dawoo\\Desktop\\predict.jpg")  # 예측된 이미지를 저장



image 1/1 C:\Users\dawoo\Desktop\1.jpg: 448x640 1 faulty, 32.3ms
Speed: 6.0ms preprocess, 32.3ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
boxes : tensor([[280.9799, 172.6270, 172.4541, 240.4579]], device='cuda:0')


'C:\\Users\\dawoo\\Desktop\\predict.jpg'

In [ ]:
print("results 길이:", len(results))
print("test_test_20 길이:", len(test_test_20))


In [ ]:
import cv2
from ultralytics import YOLO

# YOLOv8 모델 로드
model_path = "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_11_s_200_12_576.pt"  # 학습한 모델 경로
model = YOLO(model_path)
model.to("cuda")

# 카메라 열기
cap = cv2.VideoCapture(0)  # 기본 카메라 (ID 0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

# 카메라 프레임 크기 설정 (선택 사항)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 색상 설정 (바운딩 박스와 텍스트)
box_color = (0, 255, 0)  # 녹색
text_color = (255, 255, 255)  # 흰색

print("Press 'q' to exit...")

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break
    flip_frame = cv2.flip(frame, 1)
    # 모델 추론 (YOLOv8)
    results = model.predict(source=frame, conf=0.5, show=False,
                    verbose=False)  # 이미지에 대해 추론
    detections = results[0]  # 첫 번째 결과 가져오기

    # 바운딩 박스와 라벨 그리기
    for box in detections.boxes:
        # 바운딩 박스 좌표 (정규화된 값 아님)
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌상단(x1, y1), 우하단(x2, y2)
        conf = box.conf[0]  # 신뢰도
        cls = int(box.cls[0])  # 클래스 ID
        label = f"{model.names[cls]} {conf:.2f}"  # 클래스 이름과 신뢰도

        # 바운딩 박스와 라벨 추가
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)  # 바운딩 박스
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

    # 프레임 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", flip_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()

Press 'q' to exit...


## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')